In [3]:
import pandas as pd
import spacy
import os
import glob

In [2]:
# Define the path to the folder containing the text files
text_folder_path = os.path.join(os.getcwd(), "train-text")

# Get a list of all text files in the folder
text_files = glob.glob(os.path.join(text_folder_path, "*.txt"))

# Load text files
texts = [open(file, 'r', encoding='utf-8').read() for file in text_files]

In [5]:
# Load German NLP model
nlp = spacy.load("de_core_news_sm")

# Preprocess texts
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

In [48]:
ROOM_KEYWORDS = [
    "Schlafzimmer", "Küche", "Wohnzimmer", "Badezimmer", "Arbeitszimmer", "Gästezimmer", "Esszimmer", "Kinderzimmer", 
    "Hauptschlafzimmer", "Speisekammer", "Fitnessraum", "Waschraum", "Flur", "Terrasse", "Balkon", "Keller", "Dachboden", 
    "Garage", "Lagerraum", "Spielzimmer", "Heimkino", "Büro", "Sauna", "Gartenhaus", "Hobbyraum", "Wintergarten", 
    "Empfangsraum", "Bibliothek", "Abstellraum", "Medienraum", "Werkstatt", "Technikraum", "Galerie", "Schrankraum", 
    "Kinderbereich", "Gäste-WC", "Dusche", "Umkleideraum", "Weinkeller", "Entspannungsraum", "Lesezimmer", "Pflegezimmer", 
    "Veranda", "Begehbarer Kleiderschrank", "Treppenhaus", "Hauptbadezimmer", "Spielraum", "Offener Wohnbereich", 
    "Doppelgarage", "Multifunktionsraum", "Dachterrasse", "Haupthalle", "Besprechungsraum", "Pförtnerloge", 
    "Außenküche", "Atelier", "Leseecke", "Arbeitsfläche", "Aufenthaltsraum", "Lounge", "Veranstaltungsraum", 
    "Cafeteria", "Bar", "Pavillon", "Behandlungsraum", "Eingangsbereich", "Meditationsraum", "Bastelraum", 
    "Fitnessstudio", "Aufnahmeraum", "Serverraum", "Klassenraum", "Lagerfläche", "Übergangsbereich", "Heizungsraum", 
    "Labor", "Küchenzeile", "Arbeitsinsel", "Speiseraum", "Pausenraum", "Schlafsaal", "Ruheraum", "Tanzstudio", 
    "Empfangsbereich", "Arbeitsstation", "Werkraum", "Außenraum", "Gruppenraum", "Kunststudio", "Rezeption", 
    "Konferenzraum", "Bewegungsraum"
]


SPECIAL_FEATURES_KEYWORDS = [
    "Whirlpool", "Pool", "Sauna", "Terrasse", "Balkon", "Garten", "Feuerstelle", "Kamin", "Fitnessraum", "Heimkino", 
    "Weinkeller", "Außenküche", "Wintergarten", "Veranda", "Bibliothek", "Gewächshaus", "Sichtfenster", "Skylights", 
    "Jacuzzi", "Grillplatz", "Spielplatz", "Trampolin", "Automatisierung", "Smart-Home-System", "Schiebedach", 
    "Lichtkuppel", "Kletterwand", "Fischteich", "Vogelhaus", "Wasserfall", "Kinderspielplatz", "Baumhaus", 
    "Fahrradständer", "Solarpaneele", "Ladestation für Elektroautos", "Whiskeyzimmer", "Billardraum", "Partykeller", 
    "Tanzfläche", "Karaokeraum", "Yoga-Studio", "Beobachtungsdeck", "Klettergerüst", "Boccia-Bahn", "Bowlingbahn", 
    "Tennisplatz", "Golfplatz", "Laufband", "Wellnessbereich", "Behandlungsraum", "Trainingsfläche", 
    "Privater Strand", "Strandkorb", "Indoor-Garten", "Bilderrahmenwand", "Leseecke", "Spielzimmer", "Hobbyraum", 
    "Beamerraum", "Poolhaus", "Offener Kamin", "Naturteich", "Winterlandschaft", "Weinlager", "Gartenschuppen", 
    "Außenfeuerstelle", "Glaswand", "Eingebauter Grill", "Panoramablick", "Wasserfontäne", "Außenbeleuchtung", 
    "Gartenteich", "Kinderrutsche", "Baumhaus", "Außenlounge", "Liegefläche", "Freiluftdusche", "Fass-Sauna", 
    "Natursteinmauer", "Kneippbecken", "Indoor-Pool", "Spielgeräte", "Gartenmöbel", "Pergola", "Hängematte", 
    "Kletterpflanzen", "Kräutergarten", "Ziergarten", "Vogeltränke", "Swimmingpool"
]

DESIGN_STYLE_KEYWORDS = [
    "modern", "rustikal", "minimalistisch", "skandinavisch", "industriell", "klassisch", "traditionell", 
    "zeitgenössisch", "barock", "boho", "luxuriös", "gemütlich", "elegant", "natürlich", "alpin", "ländlich", 
    "antik", "viktorianisch", "kolonialstil", "chalet-stil", "mediterran", "orientalisch", "japanisch", 
    "mid-century modern", "art déco", "futuristisch", "maritim", "loft", "eklektisch", "griechisch", 
    "georgianisch", "kubistisch", "tropisch", "industrial chic", "französisch", "marokkanisch", "scandi-chic", 
    "california style", "landhausstil", "natural chic", "urban modern", "contemporary classic", "retro", 
    "neo-gothisch", "avantgarde", "provencial", "veranda style", "ethnisch", "postmodern", "high-tech", 
    "new-classic", "zen", "country-modern", "kanadisch", "alpenstil", "bohemian", "vintage", "pariser chic", 
    "gartenstil", "blockhausstil", "asiatisch", "coastal chic", "grünraumdesign", "kulturstil", "bali-stil", 
    "italienisch", "australisch", "neoklassisch", "seemannsstil", "alhambra-stil", "organisch", "cottagecore", 
    "hacienda-stil", "norwegisch", "schwedisch", "irisch", "holländisch", "luxury loft"
]

MATERIALS_KEYWORDS = [
    "Holz", "Beton", "Stahl", "Glas", "Ziegel", "Marmor", "Granit", "Kunststoff", "Kupfer", "Aluminium", 
    "Keramik", "Fliesen", "Schiefer", "Naturstein", "Laminat", "PVC", "Lehm", "Bambus", "Eichenholz", 
    "Kiefernholz", "Recyclingmaterialien", "Verbundwerkstoffe", "Carbonfaser", "Fertigbeton", "Kalksandstein", 
    "Faserzement", "Sperrholz", "Edelstahl", "Weichholz", "Hartholz", "Metall", "Kunstharz", "Terrakotta", 
    "Zement", "Dämmstoffe", "Gipskarton", "Porenbeton", "Wellblech", "Furnier", "Gusseisen", "Polycarbonat", 
    "Akustikplatten", "Isolierglas", "Solarzellen", "Quarzit", "Travertin", "Kork", "Basalt", "Schlackestein", 
    "Polymere", "Asphalt", "Dachziegel", "Stroh", "Zementestrich", "Tonziegel", "Eisen", "Textilbeton", 
    "Filz", "Vinyl", "Bitumen", "Styropor", "Glasfaser", "Elastomer", "Papier", "Naturfasern", "Leder", 
    "Biokomposite", "Pressholz", "Epoxidharz", "Hanf", "Jute", "Seide", "Schieferplatten", "Ton", "Blech", 
    "Titan", "Gummimaterialien", "Glaswolle", "Perlite", "Bimsstein", "Polyethylen", "Kunstleder", 
    "Kunstmarmor", "Schweißdraht", "Holzwerkstoffplatten", "Styrodur", "Lichtbeton", "Pigmentierte Farben", 
    "Keramische Fliesen", "Naturmaterialien", "Säulenstein", "Architektenbeton", "Leichtbeton"
]

BUDGET_KEYWORDS = [
    "Euro", "Kosten", "Budget", "Preis", "Finanzierung", "Baukosten", "Projektkosten", "Schätzung", 
    "Investition", "Kostenrahmen", "Kostenvoranschlag", "Bausumme", "Endsumme", "Geldmittel", "Ausgaben", 
    "Aufwand", "Eigenkapital", "Gesamtbudget", "Planungskosten", "Baukredit", "Rohbaukosten", "Materialkosten", 
    "Lohnkosten", "Baufinanzierung", "Kreditrahmen", "Bauzinsen", "Gebühren", "Fördermittel", "Subventionen", 
    "Monatsraten", "Baukreditkosten", "Versicherungskosten", "Rücklagen", "Anzahlung", "Teilsumme", 
    "Gesamtaufwand", "Projektfinanzierung", "Bauabgaben", "Zuschuss", "Abschreibungskosten", "Umsatzsteuer", 
    "Fixkosten", "Vertragssumme", "Abrechnungsbetrag", "Voranschlag", "Eigenleistungskosten", "Planungspuffer", 
    "Projektschätzung", "Baurechnung", "Vorauszahlung", "Steuerkosten", "Gebäudewert", "Schuldendienst", 
    "Sonderkosten", "Kalkulation", "Kreditkosten", "Leasingrate", "Bauzuschuss", "Ersparnisse", "Kapitalbedarf", 
    "Finanzierungsplan", "Abzahlungsplan", "Kostenschätzung", "Projektmittel", "Tilgungsplan", "Hypothek", 
    "Baukostenanalyse", "Kreditbewilligung", "Eigenkapitalquote", "Mittelzuweisung", "Projektfonds", 
    "Rückzahlungsbetrag", "Gewährleistungskosten", "Baukostenübersicht", "Abrechnungsplan", "Schuldzins", 
    "Hauswert", "Bauabsicherung", "Tilgungsbeitrag", "Baukreditangebot", "Bauversicherungsbeitrag", 
    "Baubuchhaltung", "Baumittel", "Zinskosten", "Nebenkosten", "Baudarlehen", "Gesamtausgaben", 
    "Zinsaufwand", "Baupreis", "Materialpreis", "Honorarkosten", "Vertragskosten", "Vertragswert"
]

TIMELINE_KEYWORDS = [
    "Monate", "Jahre", "Wochen", "Zeitrahmen", "Fertigstellung", "Abschluss", "Projektzeit", "Bauzeit", 
    "Termine", "Planungsphase", "Zeitplan", "Meilensteine", "Lieferdatum", "Bauabschluss", "Abgabetermin", 
    "Fertigstellungsdatum", "Frist", "Baukalender", "Projektkalender", "Projektfristen", "Starttermin", 
    "Bauprojektzeit", "Projektstart", "Zieltermin", "Abschlusstermin", "Zeiteinschätzung", "Baufristen", 
    "Projektdauer", "Umsetzungszeit", "Auftragsdauer", "Bearbeitungszeit", "Planungstermine", "Ausführungsphase", 
    "Konstruktionszeit", "Baufertigstellung", "Abnahmefrist", "Lieferfristen", "Verzögerungen", "Endzeit", 
    "Baulaufzeit", "Projektstartdatum", "Projektenddatum", "Zeitabschätzung", "Arbeitszeit", "Projektzyklus", 
    "Bauetappen", "Bauphasen", "Bauverzug", "Projektetappen", "Durchlaufzeit", "Terminvorgabe", "Fertigstellungsfristen", 
    "Fertigstellungsphase", "Zeitschiene", "Bauprojektfristen", "Projektfortschritt", "Vorlaufzeit", 
    "Termineinhaltung", "Abschlusstermine", "Zeithorizont", "Startzeitpunkt", "Endzeitpunkt", "Tagesplan", 
    "Wochenplan", "Zeitmanagement", "Zeitpuffer", "Projektverlauf", "Projektzeitachse", "Bauvorlauf", 
    "Projektverzögerung", "Fertigstellungsdauer", "Projektdeadline", "Fristverlängerung", "Bauende", 
    "Projektende", "Bauabnahme", "Zeitfenster", "Projektmeilensteine", "Projektaufgaben", "Bauplanung", 
    "Lieferzeiträume", "Bautermin", "Zeitvorgaben", "Verzugszeit", "Projektplan", "Zeitrahmenplanung"
]

ADDITIONAL_NOTES_KEYWORDS = [
    "funktional", "nachhaltig", "energieeffizient", "familienfreundlich", "barrierefrei", "umweltfreundlich", 
    "sicher", "modular", "flexibel", "wetterbeständig", "energieautark", "anpassungsfähig", "pflegeleicht", 
    "wartungsarm", "hochwertig", "komfortabel", "innovativ", "raumoptimiert", "modernisiert", "intelligent", 
    "wartungsfreundlich", "ökologisch", "stabil", "langfristig", "ästhetisch", "luxuriös", "minimalistisch", 
    "schalldicht", "lichtdurchflutet", "pflegearm", "zeitgemäß", "belüftet", "hitzeresistent", "schallisoliert", 
    "rauchfrei", "grün", "sicherheitsoptimiert", "praktisch", "stilvoll", "familiengerecht", "einfallsreich", 
    "robust", "widerstandsfähig", "technologisch", "inspirierend", "benutzerfreundlich", "kostenbewusst", 
    "platzsparend", "geräuscharm", "intelligent", "belastbar", "energiesparend", "modular", "hochwertig", 
    "nachhaltigkeit", "autark", "modernisierung", "effizient", "individuell", "umweltgerecht", "pflegeleicht", 
    "langlebig", "naturnah", "gesundheitlich", "vorausschauend", "klimagerecht", "resilient", "raumoptimierung", 
    "innovativer ansatz", "freundlich", "geschmackvoll", "erweiterbar", "harmonisch", "zeitlos", "praktikabel", 
    "schadstoffarm", "dynamisch", "zukunftssicher", "resourcenschonend", "multifunktional", "adaptiv", 
    "wohnfreundlich", "ergonomisch", "modernste technik", "smart", "resistent", "wohnlich"
]

In [49]:
keywords_dict = {
    "ROOM_KEYWORDS" : ROOM_KEYWORDS,
    "SPECIAL_FEATURES_KEYWORDS" : SPECIAL_FEATURES_KEYWORDS,
    "DESIGN_STYLE_KEYWORDS" : DESIGN_STYLE_KEYWORDS,
    "MATERIALS_KEYWORDS" : MATERIALS_KEYWORDS,
    "BUDGET_KEYWORDS" : BUDGET_KEYWORDS,
    "TIMELINE_KEYWORDS" : TIMELINE_KEYWORDS,
    "ADDITIONAL_NOTES_KEYWORDS" : ADDITIONAL_NOTES_KEYWORDS
}

In [9]:
processed_keywords_dict = {
    key: [preprocess(text) for text in value_list]
    for key, value_list in keywords_dict.items()
}

In [44]:
import pandas as pd
import re

# Function to extract data with adjectives
def extract_details_german_with_adjectives(text):
    details = {
        "Project description": "",
        "Rooms": "",
        "Special features": "",
        "Design style": "",
        "Materials": "",
        "Budget": "",
        "Timeline": "",
        "Additional Notes": ""
    }
    
    # Parse the text
    details["Project description"] = text.split('\n')[0].strip()
    
    
    # Define the list of word-numbers
    word_numbers = ["ein", "eine", "einen", "zwei", "drei", "vier", "fünf", "sechs", "sieben", "acht", "neun", "zehn"]
    number_pattern = rf"(\b(?:\d+|{'|'.join(word_numbers)})\b)"  # Match numeric or word-numbers
    # Rooms with adjectives or numbers
    room_keywords = "|".join(keywords_dict["ROOM_KEYWORDS"])
    rooms_pattern = rf"{number_pattern}?\s*(\b[\wäöüß]+\b)?\s*({room_keywords})"
    rooms_matches = re.findall(rooms_pattern, text, re.IGNORECASE)
    # Format the results
    details["Rooms"] = ", ".join([
        f"{num or ''} {adj or ''} {room}".strip() 
        for num, adj, room in rooms_matches if room
    ])
    
    
    # Special Features with adjectives
    features_keywords = "|".join(keywords_dict["SPECIAL_FEATURES_KEYWORDS"])  # Join keywords into a regex pattern
    features_pattern = rf"(\b[\wäöüß]+\b)?\s*({features_keywords})"  # Use rf-string for dynamic pattern creation
    features_matches = re.findall(features_pattern, text, re.IGNORECASE)

    # Format the matched features
    details["Special features"] = ", ".join([
        f"{adj or ''} {feature}".strip() for adj, feature in features_matches if feature
    ])
 
    # Design Style
    style_keywords = "|".join(keywords_dict["DESIGN_STYLE_KEYWORDS"])  # Join keywords into a regex pattern
    style_pattern = rf"({style_keywords})"  # Use rf-string for interpolation
    style_match = re.search(style_pattern, text, re.IGNORECASE)
    details["Design style"] = style_match.group(0) if style_match else ""

    # Materials
    material_keywords = "|".join(keywords_dict["MATERIALS_KEYWORDS"])  # Join keywords into a regex pattern
    material_pattern = rf"({material_keywords})"  # Use rf-string for interpolation
    materials_match = re.search(material_pattern, text, re.IGNORECASE)
    details["Materials"] = materials_match.group(0) if materials_match else ""
        
    # Budget
    budget_pattern = r"([\d\s.,]+(?:€|Euro))"
    budget_match = re.search(budget_pattern, text, re.IGNORECASE)
    #budget_match = re.search(r"(\d+(?:,\d+)?(?: €| Euro))", text, re.IGNORECASE)
    details["Budget"] = budget_match.group(0) if budget_match else ""
    
    # Timeline
# Timeline
    timeline_pattern = r"(\d+ (?:Monat(?:en|e)?|Jahr(?:en|e)?|Woche(?:n)?))"
    timeline_match = re.search(timeline_pattern, text, re.IGNORECASE)
    details["Timeline"] = timeline_match.group(0) if timeline_match else ""
    
    # Additional Notes
    notes_keywords = "|".join(keywords_dict["ADDITIONAL_NOTES_KEYWORDS"])
    notes_pattern = rf"\b(?:{notes_keywords})\b"
    notes_matches = re.findall(notes_pattern, text, re.IGNORECASE)
    details["Additional Notes"] = "|".join(notes_matches) if notes_matches else ""
    
    return details



In [24]:
texts

['Wir suchen ein großes Holzhaus für Mehrgenerationen-Wohnen. Es sollte sechs Schlafzimmer, zwei Küchen und mehrere Wohnbereiche haben, um allen gerecht zu werden. Wir möchten einen großen Garten für Aktivitäten im Freien, einen Spielbereich für Kinder und ein solarbetriebenes Heizungssystem.\n\nDas Design sollte rustikal, aber modern sein, um zu einer großen Familie zu passen, mit langlebigen und umweltfreundlichen Materialien. Unser Budget liegt bei 1.200.000 €, und wir möchten, dass es innerhalb von 22 Monaten fertiggestellt wird.',
 'Wir möchten ein modernes luxuriöses Holzhaus für Wochenendtrips. Es sollte drei Schlafzimmer, einen offenen Wohn- und Essbereich und ein Heimkino haben. Zusätzliche Merkmale sollten ein Infinity-Pool, ein Außenkamin und bodentiefe Fenster sein, um die Aussicht zu genießen.\n\nDas Design sollte zeitgenössisch mit hochwertigen Ausstattungen sein, mit Premium-Holz- und Glasmaterialien. Unser Budget beträgt 1.000.000 €, und wir hoffen, das Projekt innerhal

In [51]:
# Data extraction
data = []
for text in texts:
    #cleaned_text = preprocess(text)
    details = extract_details_german_with_adjectives(text)
    data.append(details)

# Convert to DataFrame
df = pd.DataFrame(data)

df



,Project description,Rooms,Special features,Design style,Materials,Budget,Timeline,Additional Notes
0,Wir suchen ein großes Holzhaus für Mehrgenerat...,"sechs Schlafzimmer, zwei Küche",großen Garten,rustikal,Holz,1.200.000 €,22 Monaten,
1,Wir möchten ein modernes luxuriöses Holzhaus f...,"drei Schlafzimmer, ein Heimkino","ein Heimkino, Pool, kamin",modern,Holz,1.000.000 €,18 Monaten,
2,Wir planen eine kompakte autarke Hütte für das...,"ein Schlafzimmer, eine kleine Küche",,gemütlich,,200.000 €,8 Monaten,Nachhaltigkeit|minimalistisch|nachhaltig
3,Wir planen eine luxuriöse Holzvilla für Verans...,"fünf Schlafzimmer, ein großes Wohnzimmer, eine...","einen Swimmingpool, großen Garten",luxuriös,Holz,1.500.000 €,24 Monaten,
4,"Wir möchten ein kompaktes Holzhaus, das den Pl...","zwei Schlafzimmer, funktionale Küche, eine übe...",überdachte Terrasse,gemütlich,Holz,300.000 €,10 Monate,
5,"Wir suchen ein Ferienholzhaus, das einfach und...","drei Schlafzimmer, ein gemütliches Wohnzimmer,...","große Terrasse, gerne Solarpaneele",elegant,holz,350.000 €,12 Monaten,Nachhaltigkeit
6,Wir planen ein modernes Holzhaus mit intellige...,"vier Schlafzimmer, ein Arbeitszimmer",,modern,Holz,800.000 €,18 Monate,energieeffizient
7,"Guten Morgen! Ich suche ein gemütliches, aber ...","zwei Schlafzimmer, offene Küche, der Terrasse","einen Whirlpool, der Terrasse",gemütlich,Holz,500.000 Euro,18 Monaten,luxuriös|funktional
8,Hallo! Ich möchte ein traditionelles Holzhaus ...,"drei Schlafzimmer, ein geräumiges Wohnzimmer, ...","mit Kamin, einer Sauna",traditionell,Holz,600.000 Euro,14 Monaten,langlebig
9,"Hallo! Wir planen ein Holz-Familienhaus, das s...","fünf Schlafzimmer, eine große Küche, ein Spiel...","ein Spielzimmer, wie Solarpaneele",modern,Holz,750.000 Euro,20 Monaten,funktional|umweltfreundlich


In [47]:
print(df['Project description'][3])
print(df['Rooms'][3])

Wir planen eine luxuriöse Holzvilla für Veranstaltungen. Sie sollte fünf Schlafzimmer, ein großes Wohnzimmer, eine große Küche und ein formelles Esszimmer haben. Wir möchten auch einen Swimmingpool und einen großen Garten.
fünf Schlafzimmer, ein großes Wohnzimmer, eine große Küche, ein formelles Esszimmer


In [ ]:
# Save to CSV
output_file = "/mnt/data/project_details_german_with_adjectives.csv"
df.to_csv(output_file, index=False)

In [11]:
print("|".join(keywords_dict["ROOM_KEYWORDS"]))

Schlafzimmer|Küche|Wohnzimmer|Badezimmer|Arbeitszimmer|Gästezimmer|Esszimmer|Kinderzimmer|Hauptschlafzimmer|Speisekammer|Fitnessraum|Waschraum|Flur|Terrasse|Balkon|Keller|Dachboden|Garage|Lagerraum|Spielzimmer|Heimkino|Büro|Sauna|Gartenhaus|Hobbyraum|Wintergarten|Empfangsraum|Bibliothek|Abstellraum|Medienraum|Werkstatt|Technikraum|Galerie|Schrankraum|Kinderbereich|Gäste-WC|Dusche|Umkleideraum|Weinkeller|Entspannungsraum|Lesezimmer|Pflegezimmer|Veranda|Begehbarer Kleiderschrank|Treppenhaus|Hauptbadezimmer|Spielraum|Offener Wohnbereich|Doppelgarage|Multifunktionsraum|Dachterrasse|Haupthalle|Besprechungsraum|Pförtnerloge|Außenküche|Atelier|Leseecke|Arbeitsfläche|Aufenthaltsraum|Lounge|Veranstaltungsraum|Cafeteria|Bar|Pavillon|Behandlungsraum|Eingangsbereich|Meditationsraum|Bastelraum|Fitnessstudio|Aufnahmeraum|Serverraum|Klassenraum|Lagerfläche|Übergangsbereich|Heizungsraum|Labor|Küchenzeile|Arbeitsinsel|Speiseraum|Pausenraum|Schlafsaal|Ruheraum|Tanzstudio|Empfangsbereich|Arbeitsstation|W